# Noise Tests

In [1]:
import numpy as np
from pprint import pprint

import qiskit
from qiskit import QISKitError
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

# Import Aer
import qiskit_aer
from qiskit_aer.backends import QasmSimulator

## Noise Module

In [2]:
from qiskit_aer import noise
from qiskit_aer.noise import NoiseModel, QuantumError, ReadoutError
help(noise)

Help on package qiskit_aer.noise in qiskit_aer:

NAME
    qiskit_aer.noise - Noise module for Qiskit Aer.

DESCRIPTION
    This module contains classes and functions to build a noise model for
    simulating a Qiskit quantum circuit in the presence of errors.
    
    The main noise model class is `NoiseModel` class. Errors are represented
    using the `QuantumError` and `ReadoutError` classes and can be added to a
    noise model to sample errors during specific circuit operations of a
    simulation.
    
    noise  (package)
    |
    |-- Noise Model (class)
    |-- QuantumError (class)
    |-- ReadoutError (class)
    |-- errors (package)

PACKAGE CONTENTS
    aernoiseerror
    errors (package)
    noise_model
    noise_utils
    quantum_error
    readout_error

FILE
    /Users/cjwood/git/qiskit-aer/aer/qiskit_aer/noise/__init__.py




## Errors Module

In [3]:
help(noise.errors)

Help on package qiskit_aer.noise.errors in qiskit_aer.noise:

NAME
    qiskit_aer.noise.errors - Standard error module for Qiskit Aer.

DESCRIPTION
    This module contains functions to generate QuantumError objects for
    standard noise channels in quantum information science.
    
    errors (package)
    |
    |-- mixed_unitary_error
    |-- coherent_unitary_error
    |-- pauli_error
    |-- depolarizing_error
    |-- thermal_relaxation_error
    |-- phase_amplitude_damping_error
    |-- amplitude_damping_error
    |-- phase_damping_error

PACKAGE CONTENTS
    standard_errors

FILE
    /Users/cjwood/git/qiskit-aer/aer/qiskit_aer/noise/errors/__init__.py




## Test Circuit

In [4]:
# Test circuit
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
circ = QuantumCircuit(qr, cr)
circ.h(qr[0])
circ.cx(qr[0], qr[1])
circ.measure(qr, cr)

### Ideal Simulation

In [5]:
# Ideal Backend
backend = QasmSimulator()
job = qiskit.execute(circ, backend, shots=1000)
result = job.result()
print(result.get_counts(circ))

{'0x0': 499, '0x3': 501}


### Depolarizing Channel Noise

In [6]:
from qiskit_aer.noise.errors import depolarizing_error

# Errors
error1 = depolarizing_error(1, 1, as_matrix=False)
print(error1, "\n")
error2 = depolarizing_error(1, 2, as_matrix=False)
print(error2, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error1, "h")
noise_model.add_quantum_error(error2, "cx", [0, 1])
print(noise_model)
# Backend
backend = QasmSimulator()
backend.set_noise_model(noise_model)
job = qiskit.execute(circ, backend, shots=1000)
result = job.result()
print("\nCounts:")
print(result.get_counts(circ))

QuantumError on 1 qubits. Noise circuits:
  P(0) = 0.25, circuit = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.25, circuit = [[{'name': 'y', 'qubits': [0]}]
  P(2) = 0.25, circuit = [[{'name': 'z', 'qubits': [0]}]
  P(3) = 0.25, circuit = [[{'name': 'id', 'qubits': [0]}] 

QuantumError on 2 qubits. Noise circuits:
  P(0) = 0.0625, circuit = [[{'name': 'x', 'qubits': [1]}]
  P(1) = 0.0625, circuit = [[{'name': 'y', 'qubits': [1]}]
  P(2) = 0.0625, circuit = [[{'name': 'z', 'qubits': [1]}]
  P(3) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}]
  P(4) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(5) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}, {'name': 'y', 'qubits': [1]}]
  P(6) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}, {'name': 'z', 'qubits': [1]}]
  P(7) = 0.0625, circuit = [[{'name': 'y', 'qubits': [0]}]
  P(8) = 0.0625, circuit = [[{'name': 'y', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(9) = 0.0625, circuit = [[{'nam

### X-90 based deplarizing error model

In [7]:
# Errors
error1 = depolarizing_error(1, 1, as_matrix=False)
print(error1, "\n")
error2 = depolarizing_error(1, 2, as_matrix=False)
print(error2, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.set_x90_single_qubit_gates(['h'])
noise_model.add_all_qubit_quantum_error(error1, "x90")
noise_model.add_quantum_error(error2, "cx", [0, 1])

print(noise_model)
# Backend
backend = QasmSimulator()
backend.set_noise_model(noise_model)
job = qiskit.execute(circ, backend, shots=1000)
result = job.result()
print("\nCounts:")
print(result.get_counts(circ))

QuantumError on 1 qubits. Noise circuits:
  P(0) = 0.25, circuit = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.25, circuit = [[{'name': 'y', 'qubits': [0]}]
  P(2) = 0.25, circuit = [[{'name': 'z', 'qubits': [0]}]
  P(3) = 0.25, circuit = [[{'name': 'id', 'qubits': [0]}] 

QuantumError on 2 qubits. Noise circuits:
  P(0) = 0.0625, circuit = [[{'name': 'x', 'qubits': [1]}]
  P(1) = 0.0625, circuit = [[{'name': 'y', 'qubits': [1]}]
  P(2) = 0.0625, circuit = [[{'name': 'z', 'qubits': [1]}]
  P(3) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}]
  P(4) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(5) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}, {'name': 'y', 'qubits': [1]}]
  P(6) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}, {'name': 'z', 'qubits': [1]}]
  P(7) = 0.0625, circuit = [[{'name': 'y', 'qubits': [0]}]
  P(8) = 0.0625, circuit = [[{'name': 'y', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(9) = 0.0625, circuit = [[{'nam

### Depolarizing noise from Kraus matrices

In [8]:
# We define a unitary channel as a Kraus set
# Note that internally this will be converted to a mixed unitary error channel

I = np.eye(2, dtype=complex)
X = np.array([[0, 1], [1, 0]], dtype=complex)
Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
Z = np.array([[1, 0], [0, -1]], dtype=complex)

# Errors
kraus_ops1 = [np.sqrt(0.25) * I, np.sqrt(0.25) * X, np.sqrt(0.25) * Y, np.sqrt(0.25) * Z]
error1 = QuantumError(kraus_ops1)
print(error1, "\n")

kraus_ops2 = [np.kron(a,b) for a in kraus_ops1 for b in kraus_ops1]
error2 = QuantumError(kraus_ops2)
print(error2, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error1, "h")
noise_model.add_all_qubit_quantum_error(error2, "cx")
print(noise_model, "\n")

backend = QasmSimulator()
backend.set_noise_model(noise_model)
job = qiskit.execute(circ, backend, shots=1000)
result = job.result()

print("Counts:")
print(result.get_counts(circ))

QuantumError on 1 qubits. Noise circuits:
  P(0) = 0.25, circuit = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.25, circuit = [[{'name': 'y', 'qubits': [0]}]
  P(2) = 0.25, circuit = [[{'name': 'z', 'qubits': [0]}]
  P(3) = 0.25, circuit = [[{'name': 'id', 'qubits': [0]}] 

QuantumError on 2 qubits. Noise circuits:
  P(0) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}]
  P(1) = 0.0625, circuit = [[{'name': 'y', 'qubits': [0]}]
  P(2) = 0.0625, circuit = [[{'name': 'z', 'qubits': [0]}]
  P(3) = 0.0625, circuit = [[{'name': 'x', 'qubits': [1]}]
  P(4) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(5) = 0.0625, circuit = [[{'name': 'y', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(6) = 0.0625, circuit = [[{'name': 'z', 'qubits': [0]}, {'name': 'x', 'qubits': [1]}]
  P(7) = 0.0625, circuit = [[{'name': 'y', 'qubits': [1]}]
  P(8) = 0.0625, circuit = [[{'name': 'x', 'qubits': [0]}, {'name': 'y', 'qubits': [1]}]
  P(9) = 0.0625, circuit = [[{'nam

### Classical readout error

In [9]:
# Error
P = [[0.9, 0.1], [0.3, 0.7]]
error = ReadoutError(P)
print(error, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_all_qubit_readout_error(error)
print(noise_model, "\n")

# Backend
backend = QasmSimulator()
backend.set_noise_model(noise_model)
job = qiskit.execute(circ, backend, shots=1000)
result = job.result()
print("Counts:")
print(result.get_counts(circ))

ReadoutError on 1 qubits. Assignment probabilities:
 P(j|0) =  [0.9, 0.1]
 P(j|1) =  [0.3, 0.7] 

Noise Model:
  Instructions with noise: ['measure']
  All-qubits errors: ['measure'] 

Counts:
{'0x0': 464, '0x1': 136, '0x2': 40, '0x3': 360}


### Amplitude Damping Error

In [10]:
from qiskit_aer.noise.errors import amplitude_damping_error

# Error
error = amplitude_damping_error(0.5)
print(error, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error, "h")
print(noise_model, "\n")

# Backend
backend = QasmSimulator()
backend.set_noise_model(noise_model)
job = qiskit.execute(circ, backend, shots=1000)
result = job.result()

# Results
print("Counts:")
print(result.get_counts(circ))

QuantumError on 1 qubits. Noise circuits:
  P(0) = 1.0, circuit = [[{'name': 'kraus', 'qubits': [0], 'params': [array([[1.        +0.j, 0.        +0.j],
       [0.        +0.j, 0.70710678+0.j]]), array([[0.        +0.j, 0.70710678+0.j],
       [0.        +0.j, 0.        +0.j]])]}] 

Noise Model:
  Instructions with noise: ['h']
  All-qubits errors: ['h'] 

Counts:
{'0x0': 766, '0x3': 234}


### Phase damping error

In [11]:
from qiskit_aer.noise.errors import phase_damping_error

error = phase_damping_error(0.25)
print(error, "\n")

# Noise Model
noise_model = NoiseModel()
noise_model.add_all_qubit_quantum_error(error, "h")
print(noise_model, "\n")

# Backend
backend = QasmSimulator()
backend.set_noise_model(noise_model)
job = qiskit.execute(circ, backend, shots=1000)
result = job.result()

# Counts
print("Counts:")
print(result.get_counts(circ))

QuantumError on 1 qubits. Noise circuits:
  P(0) = 1.0, circuit = [[{'name': 'kraus', 'qubits': [0], 'params': [array([[1.       +0.j, 0.       +0.j],
       [0.       +0.j, 0.8660254+0.j]]), array([[0. +0.j, 0. +0.j],
       [0. +0.j, 0.5+0.j]])]}] 

Noise Model:
  Instructions with noise: ['h']
  All-qubits errors: ['h'] 

Counts:
{'0x0': 497, '0x3': 503}
